In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/coronavirusdataset/TimeAge.csv
/kaggle/input/coronavirusdataset/Region.csv
/kaggle/input/coronavirusdataset/Time.csv
/kaggle/input/coronavirusdataset/Weather.csv
/kaggle/input/coronavirusdataset/SearchTrend.csv
/kaggle/input/coronavirusdataset/TimeProvince.csv
/kaggle/input/coronavirusdataset/TimeGender.csv
/kaggle/input/coronavirusdataset/PatientInfo.csv
/kaggle/input/coronavirusdataset/PatientRoute.csv
/kaggle/input/coronavirusdataset/SeoulFloating.csv
/kaggle/input/coronavirusdataset/Case.csv
/kaggle/input/corona-virus-brazil/brazil_covid19.csv
/kaggle/input/coronavirus-in-italy/CHANGELOG.md
/kaggle/input/coronavirus-in-italy/CODE_OF_CONDUCT_EN.md
/kaggle/input/coronavirus-in-italy/.gitignore
/kaggle/input/coronavirus-in-italy/LICENSE
/kaggle/input/coronavirus-in-italy/README.md
/kaggle/input/coronavirus-in-italy/CHANGELOG_EN.md
/kaggle/input/coronavirus-in-italy/LICENSE.xmp
/kaggle/input/coronavirus-in-italy/CODE_OF_CONDUCT.md
/kaggle/input/coronavirus-in-italy/dati-j

**This notebook is a continuation of https://www.kaggle.com/sauravmishra1710/covid-19-sars-cov-2-a-geo-statistical-analysis/**

# What is Coronavirus?

    Coronaviruses (CoV) are a large family of viruses that cause illness ranging from the common cold to more severe 
    diseases such as Middle East Respiratory Syndrome (MERS-CoV) and Severe Acute Respiratory Syndrome (SARS-CoV). 
    A novel coronavirus (nCoV) is a new strain that has not been previously identified in humans.  

    Coronaviruses are zoonotic, meaning they are transmitted between animals and people.  Detailed investigations found 
    that SARS-CoV was transmitted from civet cats to humans and MERS-CoV from dromedary camels to humans. Several 
    known coronaviruses are circulating in animals that have not yet infected humans. 

    Common signs of infection include respiratory symptoms, fever, cough, shortness of breath and breathing difficulties.
    In more severe cases, infection can cause pneumonia, severe acute respiratory syndrome, kidney failure and even death. 

    Standard recommendations to prevent infection spread include regular hand washing, covering mouth and nose when coughing
    and sneezing, thoroughly cooking meat and eggs. Avoid close contact with anyone showing symptoms of respiratory illness
    such as coughing and sneezing.


    The World Health Organization has declared the novel coronavirus outbreak a public health emergency, it has increased 
    the general fear among the public. A lot of countires have heightened their measures to fight with this virus with the 
    condition in China still senitive..More than 20 countries and territories outside of mainland China have confirmed cases 
    of the virus -- spanning Asia, Europe, North America and the Middle East -- as India, Italy and the Philippines reported 
    their first cases.



# Data Source

1. The data has been shared in kaggle @ https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset.


2. Real time data on everyday basis is also made available by the Johns Hopkins university @ 
    https://github.com/CSSEGISandData/COVID-19


# Interactive Real Time Data Visualizations

1. An Interactive real time data visualization provided by the Johns Hopkins University - https://arcg.is/0fHmTX


2. An interactive web-based dashboard to track COVID-19 in real time -    https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30120-1/fulltext


3. Real time data visuals - https://www.worldometers.info/coronavirus/


4. World Health Organization Coverage on Covid-19 - https://www.who.int/health-topics/coronavirus


# Previous Work

   Initially I worked on the same dataset in a more statistical approach. The work can be viewed @ https://www.kaggle.com/sauravmishra1710/covid-19-sars-cov-2-a-geo-statistical-analysis
   This work was more on the statistical and time series analysis. This noteboook is a continuation of the previous one and concentrates more on the geographical maps.



# Getting Started - Import Required Packages & Libraries

In [2]:
# import the necessary libraries

import numpy as np 
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
import plotly as ply
import pycountry
import folium 
from folium import plugins
import json


%config InlineBackend.figure_format = 'retina'
init_notebook_mode(connected=True)


# Utility Functions

'''Display markdown formatted output like bold, italic bold etc.'''
def formatted_text(string):
    display(Markdown(string))


'''highlight the maximum in a Series or DataFrame'''  
def highlight_max(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''), index=data.index, columns=data.columns)   

# Import Dataset

In [3]:
covid_19 = pd.read_csv('../input/corona-virus-report/covid_19_clean_complete.csv', parse_dates=['Date'])

In [4]:
print("Covid_19 Shape", covid_19.shape)

Covid_19 Shape (16874, 8)


In [5]:
covid_19.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,NaN,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


# Data pre-processing

In [6]:
formatted_text('***Covid 19 data information -***')
covid_19.info()

***Covid 19 data information -***

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16874 entries, 0 to 16873
Data columns (total 8 columns):
Province/State    7847 non-null object
Country/Region    16874 non-null object
Lat               16874 non-null float64
Long              16874 non-null float64
Date              16874 non-null datetime64[ns]
Confirmed         16874 non-null int64
Deaths            16874 non-null int64
Recovered         16874 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.0+ MB


In [7]:
formatted_text('***NULL values in the data -***')
covid_19.isnull().sum()

***NULL values in the data -***

Province/State    9027
Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
Deaths               0
Recovered            0
dtype: int64

**Initial Insights**

- The **'Province/State'** column has some missing values. We could fill in these misssing values with the 'Others' name. 
  As we do not know the State for these records, lets go with 'Others' for now.
   
- The column name for - **'Province/State'** and **'Country/Region'** can be simplified. Lets rename them to 'State' and  
  'Country' respectively.
  
- China is recorded as 'Mainland China'. We will rename it to 'China'

In [8]:
# Convert 'Last Update' column to datetime object
covid_19['Date'] = covid_19['Date'].apply(pd.to_datetime)

# Fill the missing values in 'Province/State' with the 'Country' name.
covid_19['Province/State'] = covid_19['Province/State'].replace(np.nan, covid_19['Country/Region'])

# Lets rename the columns - 'Province/State' and 'Last Update' to remove the '/' and space respectively.
covid_19.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'}, inplace=True)

# Convert 'Mainland China' to 'China'
covid_19['Country'] = np.where(covid_19['Country'] == 'Mainland China', 'China', covid_19['Country'])

# Data Glimpse
covid_19.head()

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


In [9]:
# Check for the missing values again to ensure that there are no more remaining
formatted_text('***NULL values in the data -***')
covid_19.isnull().sum()

***NULL values in the data -***

State        0
Country      0
Lat          0
Long         0
Date         0
Confirmed    0
Deaths       0
Recovered    0
dtype: int64

**==>`The data looks clean now. This should be good to continue with further analysis`**.

In [10]:
# Lets check the total #Countries affected by nCoV

formatted_text('***Affected Countries -***')
Covid_19_Countries = covid_19['Country'].unique().tolist()
print(Covid_19_Countries)
print("\n------------------------------------------------------------------")
print("\nTotal countries affected by nCoV: ",len(Covid_19_Countries))

***Affected Countries -***

['Thailand', 'Japan', 'Singapore', 'Nepal', 'Malaysia', 'Canada', 'Australia', 'Cambodia', 'Sri Lanka', 'Germany', 'Finland', 'United Arab Emirates', 'Philippines', 'India', 'Italy', 'Sweden', 'Spain', 'Belgium', 'Egypt', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan', 'Bahrain', 'Kuwait', 'Algeria', 'Croatia', 'Switzerland', 'Austria', 'Israel', 'Pakistan', 'Brazil', 'Georgia', 'Greece', 'North Macedonia', 'Norway', 'Romania', 'Estonia', 'San Marino', 'Belarus', 'Iceland', 'Lithuania', 'Mexico', 'New Zealand', 'Nigeria', 'Ireland', 'Luxembourg', 'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan', 'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal', 'Andorra', 'Latvia', 'Morocco', 'Saudi Arabia', 'Senegal', 'Argentina', 'Chile', 'Jordan', 'Ukraine', 'Hungary', 'Liechtenstein', 'Poland', 'Tunisia', 'Bosnia and Herzegovina', 'Slovenia', 'South Africa', 'Bhutan', 'Cameroon', 'Colombia', 'Costa Rica', 'Peru', 'Serbia', 'Slovakia', 'Togo', 'Malta', 'Martinique', 'Bulgaria', 'Maldives', 'Bangladesh',

So there are a total of 147 countries affected with Covid-19. 

One stand out from the above country list is the item - **`Others`**. Lets check what are these records.

In [11]:
# Now lets see the Country - 'Others' which is there in the list above
formatted_text('***Affected Country - Others***')
covid_19[covid_19['Country'] == 'Cruise Ship']

***Affected Country - Others***

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
165,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-22,0,0,0
451,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-23,0,0,0
737,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-24,0,0,0
1023,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-25,0,0,0
1309,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-26,0,0,0
1595,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-27,0,0,0
1881,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-28,0,0,0
2167,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-29,0,0,0
2453,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-30,0,0,0
2739,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-31,0,0,0


This is the **`Diamond Princess cruise ship`** which started on the **`5th of February`** from **`Yokohama, Japan`**. The ship is the biggest cluster outside China. The number of patients infected with the COVID-19 coronavirus aboard a quarantined cruise ship docked in Yokohama, Japan has continued to rise making the ship the largest cluster of the deadly virus outside China.

With nearly 6% of the 3,711 passengers and crew members now infected, the 952-foot cruise ship also has the **`highest infection rate of the coronavirus`** anywhere in the world. Wuhan, China, the city where the disease is believed to have originated has nearly 33,000 official cases—but spread across a population of more than 11 million, that’s an infection rate of less than 0.3%.

Health experts say the Diamond Princess highlights the high risk that an infection will spread in confined spaces. According to the U.S. Centers for Disease Control and Preventions, **`close-contact environments such as cruises can facilitate the transmission of viruses through droplets or contaminated surfaces`**.

There have been death cases reported from the ship. Two Japanese passengers who had been on the quarantined Diamond Princess cruise ship have died after being infected with the novel coronavirus. Japan's health ministry says the male and female passengers were hospitalized last week. They were both in their 80s.
The man and woman are the first Diamond Princess passengers to die during the virus outbreak. The cruise ship has been under a quarantine at Yokohama's port near Tokyo since Feb. 3.

Source - 

 - https://time.com/5783451/covid-19-princess-diamond-cruise-ship/
 - https://www.channelnewsasia.com/news/asia/covid19-coronavirus-diamond-princess-japan-79-test-positive-12450498
 - https://www.npr.org/sections/goatsandsoda/2020/02/20/807745305/coronavirus-2-princess-diamond-cruise-ship-passengers-die-after-contracting-covi

# Diamond Cruise Ship Data

In [12]:
# Lets create a subset of the data for the cruise ship

diamond_cruise_ship_cases = covid_19[covid_19['Country'] == 'Cruise Ship']

# Data Glimpse
diamond_cruise_ship_cases.head()

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
165,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-22,0,0,0
451,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-23,0,0,0
737,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-24,0,0,0
1023,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-25,0,0,0
1309,Diamond Princess,Cruise Ship,35.4437,139.638,2020-01-26,0,0,0


# World Data

In [13]:
# Now that we have created a different subset for the cruise ship data, lets derive a subset with only the country data
covid_19_world_data = covid_19[covid_19['Country'] != 'Cruise Ship']

formatted_text('***World Data -***')
# Data Glimpse
covid_19_world_data.head()

***World Data -***

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


In [14]:
formatted_text('***World Data Countries Afftected -***')

print(covid_19_world_data.Country.unique().tolist())
print("\nTotal number of countries: ", len(covid_19_world_data.Country.unique().tolist()))

***World Data Countries Afftected -***

['Thailand', 'Japan', 'Singapore', 'Nepal', 'Malaysia', 'Canada', 'Australia', 'Cambodia', 'Sri Lanka', 'Germany', 'Finland', 'United Arab Emirates', 'Philippines', 'India', 'Italy', 'Sweden', 'Spain', 'Belgium', 'Egypt', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan', 'Bahrain', 'Kuwait', 'Algeria', 'Croatia', 'Switzerland', 'Austria', 'Israel', 'Pakistan', 'Brazil', 'Georgia', 'Greece', 'North Macedonia', 'Norway', 'Romania', 'Estonia', 'San Marino', 'Belarus', 'Iceland', 'Lithuania', 'Mexico', 'New Zealand', 'Nigeria', 'Ireland', 'Luxembourg', 'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan', 'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal', 'Andorra', 'Latvia', 'Morocco', 'Saudi Arabia', 'Senegal', 'Argentina', 'Chile', 'Jordan', 'Ukraine', 'Hungary', 'Liechtenstein', 'Poland', 'Tunisia', 'Bosnia and Herzegovina', 'Slovenia', 'South Africa', 'Bhutan', 'Cameroon', 'Colombia', 'Costa Rica', 'Peru', 'Serbia', 'Slovakia', 'Togo', 'Malta', 'Martinique', 'Bulgaria', 'Maldives', 'Bangladesh',

In [15]:
formatted_text('***Country and State wise grouped data -***')

covid_19_country_wise_data = covid_19_world_data.groupby(['Country', 'State'])['Confirmed', 'Deaths', 'Recovered'].max()
covid_19_country_wise_data

***Country and State wise grouped data -***

,,Confirmed,Deaths,Recovered
Country,State,,,
Afghanistan,Afghanistan,24,0,1
Albania,Albania,70,2,0
Algeria,Algeria,90,11,32
Andorra,Andorra,75,0,1
Angola,Angola,1,0,0
...,...,...,...,...
Uzbekistan,Uzbekistan,33,0,0
Venezuela,Venezuela,42,0,0
Vietnam,Vietnam,91,0,16


### Latest World Data

In [16]:
strDate = covid_19_world_data['Date'][-1:].astype('str')
year = int(strDate.values[0].split('-')[0])
month = int(strDate.values[0].split('-')[1])
day = int(strDate.values[0].split('-')[2].split()[0])

formatted_text('***Last reported case date-time***')
print(strDate)
print(year)
print(month)
print(strDate.values[0].split('-')[2].split())

***Last reported case date-time***

16873    2020-03-20
Name: Date, dtype: object
2020
3
['20']


In [17]:
latest_covid_19_data = covid_19_world_data[covid_19_world_data['Date'] == pd.Timestamp(date(year,month,day))]

latest_covid_19_data.reset_index(inplace=True, drop=True)

latest_covid_19_data.head()

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Thailand,Thailand,15.0000,101.0000,2020-03-20,322,1,42
1,Japan,Japan,36.0000,138.0000,2020-03-20,963,33,191
2,Singapore,Singapore,1.2833,103.8333,2020-03-20,385,0,124
3,Nepal,Nepal,28.1667,84.2500,2020-03-20,1,0,1
4,Malaysia,Malaysia,2.5000,112.5000,2020-03-20,1030,3,87


In [18]:
CountryWiseData = pd.DataFrame(latest_covid_19_data.groupby('Country')['Confirmed', 'Deaths', 'Recovered'].sum())
CountryWiseData['Country'] = CountryWiseData.index
CountryWiseData.index = np.arange(1, len(covid_19_world_data.Country.unique().tolist())+1)

CountryWiseData = CountryWiseData[['Country','Confirmed', 'Deaths', 'Recovered']]

formatted_text('***Country wise numbers of ''Confirmed'', ''Deaths'', ''Recovered'' Cases***')

#CountryWiseData = pd.merge(latest_covid_19_data[['Country', 'Lat','Long']], CountryWiseData, on='Country')

#CountryWiseData = CountryWiseData.drop_duplicates(subset = "Country", keep = 'first', inplace = True) 

CountryWiseData

***Country wise numbers of Confirmed, Deaths, Recovered Cases***

,Country,Confirmed,Deaths,Recovered
1,Afghanistan,24,0,1
2,Albania,70,2,0
3,Algeria,90,11,32
4,Andorra,75,0,1
5,Angola,1,0,0
...,...,...,...,...
157,Uzbekistan,33,0,0
158,Venezuela,42,0,0
159,Vietnam,91,0,16
160,Zambia,2,0,0


In [19]:
# Import the WORLD Latitute Longitude Data

world_lat_lon_coordinates = pd.read_csv('/kaggle/input/world-coordinates/world_coordinates.csv')
world_lat_lon_coordinates.head()

,Code,Country,latitude,longitude
0,AD,Andorra,42.546245,1.601554
1,AE,United Arab Emirates,23.424076,53.847818
2,AF,Afghanistan,33.939110,67.709953
3,AG,Antigua and Barbuda,17.060816,-61.796428
4,AI,Anguilla,18.220554,-63.068615


In [20]:
# Merge the Country co-coordinates above to the country wise data we created.

CountryWiseData = pd.merge(world_lat_lon_coordinates, CountryWiseData, on='Country')
CountryWiseData.head()

,Code,Country,latitude,longitude,Confirmed,Deaths,Recovered
0,AD,Andorra,42.546245,1.601554,75,0,1
1,AE,United Arab Emirates,23.424076,53.847818,140,2,31
2,AF,Afghanistan,33.939110,67.709953,24,0,1
3,AG,Antigua and Barbuda,17.060816,-61.796428,1,0,0
4,AL,Albania,41.153332,20.168331,70,2,0


# Map Visualizations

# Country wise distribution

In [21]:
WorldMap = folium.Map(location=[0,0], zoom_start=1.5,tiles='cartodbpositron')

formatted_text('***Click on the pin to veiw details stats***')

for lat, long, confirmed, deaths, recovered, country in zip(CountryWiseData['latitude'],
                                                           CountryWiseData['longitude'],
                                                           CountryWiseData['Confirmed'],
                                                           CountryWiseData['Deaths'],
                                                           CountryWiseData['Recovered'], 
                                                           CountryWiseData['Country']):

    if (deaths == 0):
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               '<strong>Country:</strong> ' + str(country) + '<br>'
                               '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                               '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                               '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='darkblue',icon='info-sign'), color='rgb(55, 83, 109)'
                    , tooltip = str(country), fill_color='rgb(55, 83, 109)').add_to(WorldMap)

    else:
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               '<strong>Country:</strong> ' + str(country) + '<br>'
                               '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                               '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                               '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(country), fill_color='rgb(26, 118, 255)').add_to(WorldMap)
        
WorldMap

***Click on the pin to veiw details stats***

# Country & State wise distribution

In [22]:
WorldMap = folium.Map(location=[0,0], zoom_start=1.5,tiles='Stamen Toner')

formatted_text('***Click on the dots to veiw details stats***')

for lat, long, confirmed, deaths, recovered, country, state in zip(latest_covid_19_data['Lat'],
                                                           latest_covid_19_data['Long'],
                                                           latest_covid_19_data['Confirmed'],
                                                           latest_covid_19_data['Deaths'],
                                                           latest_covid_19_data['Recovered'], 
                                                           latest_covid_19_data['Country'],
                                                           latest_covid_19_data['State']):

    if (deaths == 0):
        folium.CircleMarker(location=[lat, long]
                    , radius=3
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               '<strong>Country:</strong> ' + str(country) + '<br>'
                               '<strong>State:</strong> ' + str(state) + '<br>'
                               '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                               '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                               '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , color='blue'
                    , tooltip = str(state)
                    , fill_color='blue'
                    , fill_opacity=0.7).add_to(WorldMap)

    else:
        folium.CircleMarker(location=[lat, long]
                    , radius=3
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               '<strong>Country:</strong> ' + str(country) + '<br>'
                               '<strong>State:</strong> ' + str(state) + '<br>'
                               '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                               '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                               '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , color='red'
                    , tooltip = str(state)
                    , fill_color='red'
                    , fill_opacity=0.7).add_to(WorldMap)
        
WorldMap

***Click on the dots to veiw details stats***

# Choropleth Global

## Confirmed Cases

In [23]:
choropleth_map_confirmed = px.choropleth(CountryWiseData, locations='Country', 
                    locationmode='country names', color='Confirmed', 
                    hover_name='Country', range_color=[1,30], 
                    color_continuous_scale='reds', 
                    title='Covid-19 Globally Confirmed Countries')

choropleth_map_confirmed.update(layout_coloraxis_showscale=False)
iplot(choropleth_map_confirmed)

**`China is the worst affected country with Covid-19. The virus has spread to other neighbouring countries and cases of covid-19 have been reported there. 
However the numbers are not as high as China. Some distant countries in Europe, North America & Australia have also seen cases of Covid-19. 
This could be due to some citizens would have been present in China at the time of the virus out-break and unknowingly would have carried along with them in 
their return journey back to their respective countries.`**

## Global Deaths 

In [24]:
choropleth_map_deaths = px.choropleth(CountryWiseData, locations='Country', locationmode='country names', color='Deaths', hover_name='Country', range_color=[1,30], 
                                      color_continuous_scale='reds', title='Covid-19 Global Deaths Numbers')

choropleth_map_deaths.update(layout_coloraxis_showscale=False)
iplot(choropleth_map_deaths)

**`As China has the most reported cases, the number of deaths has also been on the higher side. The virus did not match any other known virus. 
This raised concern because when a virus is new, we do not know how it affects people. There were no existing medications available. 
Due to lack of timely medication available the number of deaths has see the higher side`**

## Global Recovered Cases

In [25]:
choropleth_map_recovered = px.choropleth(CountryWiseData, locations='Country', 
                    locationmode='country names', color='Recovered', 
                    hover_name='Country', range_color=[1,30], 
                    color_continuous_scale='reds', 
                    title='Covid-19 Global Recovered Cases')

choropleth_map_recovered.update(layout_coloraxis_showscale=False)
iplot(choropleth_map_recovered)

**`The recovery rate has been a little slowerthan expected. The virus did not match any other known virus. 
This raised concern because when a virus is new, we do not know how it affects people. There were no existing medications available. 
However, A team of doctors in Thailand have seen some apparent success treating Coronavirus with drug cocktail. The doctors combined the anti-flu drug oseltamivir with lopinavir 
and ritonavir, anti-virals used to treat HIV, Kriengsak said, adding the ministry was awaiting research results to prove the findings.`**

Read more at: https://economictimes.indiatimes.com/news/international/world-news/thailand-sees-apparent-success-treating-virus-with-drug-cocktail/articleshow/73879572.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst

# China State wise distribution

In [26]:
chinese_data_over_time = covid_19[(covid_19['Country'] == 'China')]
chinese_data_over_time.head()

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
154,Hubei,China,30.9756,112.2707,2020-01-22,444,17,28
158,Guangdong,China,23.3417,113.4244,2020-01-22,26,0,0
159,Henan,China,33.8820,113.6140,2020-01-22,5,0,0
160,Zhejiang,China,29.1832,120.0934,2020-01-22,10,0,0
161,Hunan,China,27.6104,111.7088,2020-01-22,4,0,0


In [27]:
china_statewise_data = chinese_data_over_time.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max()

china_statewise_data['State'] = china_statewise_data.index
china_statewise_data.index = np.arange(1, len(china_statewise_data.State.unique().tolist())+1)

china_statewise_data = china_statewise_data[['State','Confirmed', 'Deaths', 'Recovered']]

formatted_text('***Country wise numbers of ''Confirmed'', ''Deaths'', ''Recovered'' Cases***')

china_statewise_data.head()

***Country wise numbers of Confirmed, Deaths, Recovered Cases***

,State,Confirmed,Deaths,Recovered
1,Anhui,990,6,984
2,Beijing,491,8,390
3,Chongqing,576,6,570
4,Fujian,299,1,295
5,Gansu,134,2,98


In [28]:
# Extract the state latitude and longitude coordinates from the time series data.
china_coordinates = chinese_data_over_time[['State','Lat','Long']]
china_coordinates.drop_duplicates(keep='first', inplace=True)

china_coordinates.index = np.arange(1, len(china_coordinates.State.unique().tolist())+1)

china_coordinates.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,State,Lat,Long
1,Hubei,30.9756,112.2707
2,Guangdong,23.3417,113.4244
3,Henan,33.8820,113.6140
4,Zhejiang,29.1832,120.0934
5,Hunan,27.6104,111.7088


In [29]:
china_statewise_data = pd.merge(china_coordinates, china_statewise_data, on='State')

china_statewise_data.head()

,State,Lat,Long,Confirmed,Deaths,Recovered
0,Hubei,30.9756,112.2707,67800,3133,58382
1,Guangdong,23.3417,113.4244,1395,8,1323
2,Henan,33.8820,113.6140,1273,22,1250
3,Zhejiang,29.1832,120.0934,1234,1,1219
4,Hunan,27.6104,111.7088,1018,4,1014


# China - Distribution on Map

In [30]:
china_lat = 35.8617
china_lon = 104.1954

formatted_text('***Click on the pin to veiw details stats***')

ChinaMap = folium.Map(location=[china_lat, china_lon], zoom_start=4, tiles='cartodbpositron')

for lat, long, confirmed, deaths, recovered, state in zip(china_statewise_data['Lat'],
                                                           china_statewise_data['Long'],
                                                           china_statewise_data['Confirmed'],
                                                           china_statewise_data['Deaths'],
                                                           china_statewise_data['Recovered'], 
                                                           china_statewise_data['State']):
    
    if (deaths == 0):
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='darkblue',icon='info-sign'), color='rgb(55, 83, 109)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(55, 83, 109)').add_to(ChinaMap)
    else:
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Confirmed:</strong> ' + str(int(confirmed)) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(26, 118, 255)').add_to(ChinaMap)
    
    
ChinaMap

***Click on the pin to veiw details stats***

# Choropleth China

In [31]:
# Load the CHina geo json file

with open('/kaggle/input/china-geo-json/china_geojson.json') as file:
    china = json.load(file)

## Confirmed Cases

In [32]:
china_conf_choropleth = go.Figure(go.Choroplethmapbox(geojson=china, locations=china_statewise_data['State'],
                                                      z=china_statewise_data['Confirmed'], colorscale='Aggrnyl',
                                                      zmin=0, zmax=10000, marker_opacity=0.5, marker_line_width=0))

china_conf_choropleth.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, 
                                    mapbox_center = {"lat": china_lat, "lon": china_lon})

china_conf_choropleth.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

iplot(china_conf_choropleth)

**`Hubei is the worst affected state in China with almost 92% of the cases reported. The virus outbreak has been feared to be started in Wuhan, Hubei`**

## Death Cases

In [33]:
china_death_choropleth = go.Figure(go.Choroplethmapbox(geojson=china, locations=china_statewise_data['State'],
                                                      z=china_statewise_data['Deaths'], colorscale='Sunset',
                                                      zmin=0, zmax=3000, marker_opacity=0.5, marker_line_width=0))

china_death_choropleth.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, 
                                    mapbox_center = {"lat": china_lat, "lon": china_lon})

china_death_choropleth.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

iplot(china_death_choropleth)

## Recovered Cases

In [34]:
china_recovered_choropleth = go.Figure(go.Choroplethmapbox(geojson=china, locations=china_statewise_data['State'],
                                                      z=china_statewise_data['Recovered'], colorscale='Brbg',
                                                      zmin=0, zmax=8000, marker_opacity=0.5, marker_line_width=0))

china_recovered_choropleth.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, 
                                    mapbox_center = {"lat": china_lat, "lon": china_lon})

china_recovered_choropleth.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

iplot(china_recovered_choropleth)

**`The recovery rate has been a little slowerthan expected. The virus did not match any other known virus. This raised concern because when a virus is new, we do not 
know how it affects people. There were no existing medications available. However, A team of doctors in Thailand have seen some apparent success treating Coronavirus with 
drug cocktail. The doctors combined the anti-flu drug oseltamivir with lopinavir and ritonavir, anti-virals used to treat HIV, Kriengsak said, adding the ministry was 
awaiting research results to prove the findings.`**

Read more at: https://economictimes.indiatimes.com/news/international/world-news/thailand-sees-apparent-success-treating-virus-with-drug-cocktail/articleshow/73879572.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst

# Rest of World

In [35]:
rest_of_world = CountryWiseData[CountryWiseData['Country'] != 'China']
rest_of_world.head()

,Code,Country,latitude,longitude,Confirmed,Deaths,Recovered
0,AD,Andorra,42.546245,1.601554,75,0,1
1,AE,United Arab Emirates,23.424076,53.847818,140,2,31
2,AF,Afghanistan,33.939110,67.709953,24,0,1
3,AG,Antigua and Barbuda,17.060816,-61.796428,1,0,0
4,AL,Albania,41.153332,20.168331,70,2,0


## Rest of World - Confirmed Cases

In [36]:
rest_of_world_confirmed = px.choropleth(rest_of_world, locations='Country', 
                    locationmode='country names', color='Confirmed', 
                    hover_name='Country', range_color=[1, 20], 
                    color_continuous_scale='Geyser', 
                    title='Covid-19 Rest of World Confirmed Cases')

iplot(rest_of_world_confirmed)

## Rest of World - Death Cases

In [37]:
rest_of_world_death = px.choropleth(rest_of_world, locations='Country', 
                    locationmode='country names', color='Deaths', 
                    hover_name='Country', range_color=[0, 1], 
                    color_continuous_scale='Picnic', 
                    title='Covid-19 Rest of World Death Cases')

iplot(rest_of_world_death)

**`There have been countable number of deaths reported outside of China.`**

## Rest of World - Recovered Cases

In [38]:
rest_of_world_recovered = px.choropleth(rest_of_world, locations='Country', 
                    locationmode='country names', color='Recovered', 
                    hover_name='Country', range_color=[1,20], 
                    color_continuous_scale='viridis', 
                    title='Covid-19 Rest of World Recovered Cases')

iplot(rest_of_world_recovered)

**`Outside of China there hve been countries where the confirmed cases have been all recovered from the virus`**

Lets see how many such countries are there - 

In [39]:
formatted_text('***Countries withh all reported cases recovered -***')
print(rest_of_world[rest_of_world['Confirmed'] == 
                    rest_of_world['Recovered']][['Country','Confirmed', 'Recovered']].reset_index())

***Countries withh all reported cases recovered -***

   index Country  Confirmed  Recovered
0    102   Nepal          1          1


# Diamond Princess Cruise Ship

In [40]:
diamond_cruise_ship_cases.reset_index(drop=True, inplace=True)

# We only need the latest data here
temp_ship = diamond_cruise_ship_cases.sort_values(by='Date', ascending=False).head(1)[['State', 'Confirmed', 
                                                                                       'Deaths', 'Recovered']]

temp_ship

,State,Confirmed,Deaths,Recovered
58,Diamond Princess,712,7,325


In [41]:
formatted_text('***Click on the pin to veiw details stats***')
cruiseMap = folium.Map(location=[diamond_cruise_ship_cases.iloc[0]['Lat'], diamond_cruise_ship_cases.iloc[0]['Long']], 
                       tiles='cartodbpositron', min_zoom=8, max_zoom=12, zoom_start=12)

folium.Marker(location=[diamond_cruise_ship_cases.iloc[0]['Lat'], diamond_cruise_ship_cases.iloc[0]['Long']],
        popup =   '<strong>Ship : ' + str(temp_ship.iloc[0]['State']) + '<br>' +
                    '<strong>Confirmed : ' + str(temp_ship.iloc[0]['Confirmed']) + '<br>' +
                    '<strong>Deaths : ' + str(temp_ship.iloc[0]['Deaths']) + '<br>' +
                    '<strong>Recovered : ' + str(temp_ship.iloc[0]['Recovered'])
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(temp_ship.iloc[0]['State']), fill_color='rgb(26, 118, 255)').add_to(cruiseMap)

cruiseMap

***Click on the pin to veiw details stats***

This is the **`Diamond Princess cruise ship`** which started on the **`5th of February`** from **`Yokohama, Japan`**. The ship is the biggest cluster outside China. The number of patients infected with the COVID-19 coronavirus aboard a quarantined cruise ship docked in Yokohama, Japan has continued to rise making the ship the largest cluster of the deadly virus outside China.

With nearly 6% of the 3,711 passengers and crew members now infected, the 952-foot cruise ship also has the **`highest infection rate of the coronavirus`** anywhere in the world. Wuhan, China, the city where the disease is believed to have originated has nearly 33,000 official cases—but spread across a population of more than 11 million, that’s an infection rate of less than 0.3%.

Health experts say the Diamond Princess highlights the high risk that an infection will spread in confined spaces. According to the U.S. Centers for Disease Control and Preventions, **`close-contact environments such as cruises can facilitate the transmission of viruses through droplets or contaminated surfaces`**.

There have been death cases reported from the ship. Two Japanese passengers who had been on the quarantined Diamond Princess cruise ship have died after being infected with the novel coronavirus. Japan's health ministry says the male and female passengers were hospitalized last week. They were both in their 80s.
The man and woman are the first Diamond Princess passengers to die during the virus outbreak. The cruise ship has been under a quarantine at Yokohama's port near Tokyo since Feb. 3.

Source - 

 - https://time.com/5783451/covid-19-princess-diamond-cruise-ship/
 - https://www.channelnewsasia.com/news/asia/covid19-coronavirus-diamond-princess-japan-79-test-positive-12450498
 - https://www.npr.org/sections/goatsandsoda/2020/02/20/807745305/coronavirus-2-princess-diamond-cruise-ship-passengers-die-after-contracting-covi

# Treemaps

In [42]:
china_statewise_data["Country"] = "China" # in order to have a single root node

fig1 = px.treemap(china_statewise_data.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
                 path=["Country", "State"], values="Confirmed", title='Number of Confirmed Cases in Chinese Provinces',
                 color_discrete_sequence = px.colors.qualitative.Prism, hover_data=["Confirmed"])

fig1.data[0].textinfo = 'label+text+value+percent entry'
py.offline.iplot(fig1)

fig2 = px.treemap(china_statewise_data.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country", "State"], values="Deaths", title='Number of Deaths Reported in Chinese Provinces',
                 color_discrete_sequence = px.colors.qualitative.Prism, hover_data=["Deaths"])

fig2.data[0].textinfo = 'label+text+value+percent entry'
py.offline.iplot(fig2)

**As we see from the above maps, Hubei, the said origin of the novel Corona Virus is the worst affected state in China with a staggering 84% of the confirmed cases coming out of it and 96% of deaths happenning there**

## Global Tree

In [43]:
temp2 = pd.DataFrame(covid_19_world_data.groupby(['Country', 'State'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index())
temp2['Global'] = "Global" # to have a single root

### Confirmed Cases

In [44]:
fig = px.treemap(temp2, 
                 path=["Global", "Country"], values="Confirmed", height=700,
                 title='Number of Confirmed Cases Around The Globe',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

fig = px.treemap(temp2, 
                 path=["Global", "Country", "State"], values="Confirmed", height=700,
                 title='Number of Confirmed Cases Around The Globe',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent parent+percent entry'
fig.show()


**Globally we have more that 2 lakhs confirmed cases of the people infectedd with the Virus from countries all overthe globe. 
The reason why WHO has declared this as a Pandemic. 
From the plots above we know -**

China (38%), Italy (17%), Iran (8%), Spain (6%), Germany (6%) are the worst affected countries followed by United States, France and South Korea.

### Reported Deaths

In [45]:
fig = px.treemap(temp2, 
                 path=["Global", "Country"], values="Deaths", height=700,
                 title='Number of Deaths reported Globally',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

fig = px.treemap(temp2, 
                 path=["Global", "Country", "State"], values="Deaths", height=700,
                 title='Number of Deaths reported Globally',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent parent+percent entry'
fig.show()

Globally thousands of people have lost their lives to the Corona virus. From the plots above we know -

China (37%), Italy (34%), Iran (13%), Spain (7%), France (2%) are the worst affected countries in terms of the number of deaths reported.

# Country Wise Analysis

## 1. Italy

Data Credits - https://github.com/pcm-dpc/COVID-19

In [46]:
Italy_Covid19 = pd.read_csv('/kaggle/input/coronavirus-in-italy/dati-regioni/dpc-covid19-ita-regioni-20200318.csv')
Italy_Covid19.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-03-18 17:00:00,ITA,13,Abruzzo,42.351222,13.398438,99,41,140,109,249,33,7,7,263,2054
1,2020-03-18 17:00:00,ITA,17,Basilicata,40.639471,15.805148,9,2,11,16,27,7,0,0,27,262
2,2020-03-18 17:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,79,18,97,269,366,84,1,9,376,2844
3,2020-03-18 17:00:00,ITA,18,Calabria,38.905976,16.594402,45,11,56,70,126,14,2,1,129,1668
4,2020-03-18 17:00:00,ITA,15,Campania,40.839566,14.250850,127,24,151,272,423,0,28,9,460,2685


**Translation**
1. deceduti = deceased
2. denominazione regione = region denomination
3. totale attualmente positivi = total currently positive
4. totale_casi = total cases
5. ricoverati con sintomi = hospitalized with symptoms
6. dimessi guariti = discharged healed

In [47]:
Italy_Covid19[['denominazione_regione','totale_attualmente_positivi','deceduti']].sort_values('totale_attualmente_positivi', ascending=False).head(5)

,denominazione_regione,totale_attualmente_positivi,deceduti
9,Lombardia,12266,1959
5,Emilia Romagna,3915,458
20,Veneto,2953,94
12,Piemonte,2187,154
10,Marche,1476,92


**Lombardia	(10043), Emilia Romagna	(2741), Veneto	(1989) are the worst affected regions in Italy.**

In [48]:
italy_lat = 42.50
italy_lon = 12.50

formatted_text('***Click on the pin to veiw details stats***')

ItalyMap = folium.Map(location=[italy_lat, italy_lon], zoom_start=6, tiles='cartodbpositron')

for lat, long, confirmed, current_positive, deaths, recovered, region in zip(Italy_Covid19['lat'],
                                                           Italy_Covid19['long'],
                                                           Italy_Covid19['totale_casi'], 
                                                           Italy_Covid19['totale_attualmente_positivi'],
                                                           Italy_Covid19['deceduti'],
                                                           Italy_Covid19['ricoverati_con_sintomi'], 
                                                           Italy_Covid19['denominazione_regione']):
    
    if (deaths == 0):
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(region).capitalize() + '<br>'
                                 '<strong>Total Confirmed:</strong> ' + str(confirmed) + '<br>'
                                 '<strong>Current Positive Cases:</strong> ' + str(int(current_positive)) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='darkblue',icon='info-sign'), color='rgb(55, 83, 109)'
                    , tooltip = str(region).capitalize(), fill_color='rgb(55, 83, 109)').add_to(ItalyMap)
    else:
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(region).capitalize() + '<br>'
                                 '<strong>Total Confirmed:</strong> ' + str(confirmed) + '<br>'
                                 '<strong>Current Positive Cases</strong> ' + str(int(current_positive)) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(region).capitalize(), fill_color='rgb(26, 118, 255)').add_to(ItalyMap)
    
    
ItalyMap

***Click on the pin to veiw details stats***

In [49]:
temp_Italy = Italy_Covid19.copy()
temp_Italy['Country'] = 'Italy' # to have a single root

fig = px.treemap(temp_Italy, 
                 path=["Country", "denominazione_regione"], values="totale_casi", height=700,
                 title='Total Confirmed Cases In Italy',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

fig = px.treemap(temp_Italy, 
                 path=["Country", "denominazione_regione"], values="totale_attualmente_positivi", height=700,
                 title='Number of Currently Positive Cases In Italy',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()


In [50]:
fig = px.treemap(temp_Italy, 
                 path=["Country", "denominazione_regione"], values="deceduti", height=700,
                 title='Number of Deaths reported in the Italian Region',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent parent+percent entry'
fig.show()

## 2. United States of America

Data Credits - https://www.kaggle.com/sudalairajkumar/covid19-in-usa

In [51]:
covid19_USA = pd.read_csv("../input/covid19-in-usa/us_states_covid19_daily.csv")
covid19_USA.head(3)

,date,state,positive,negative,pending,death,total,dateChecked
0,20200320,AK,12,686.0,NaN,NaN,698,2020-03-20T20:00:00Z
1,20200320,AL,81,28.0,NaN,0.0,109,2020-03-20T20:00:00Z
2,20200320,AR,96,351.0,203.0,NaN,650,2020-03-20T20:00:00Z


In [52]:
# US Data
covid_19_USA = covid_19[covid_19['Country'] == 'US']
covid_19_USA = covid_19_USA[covid_19_USA['State'] != 'Grand Princess']
covid_19_USA = covid_19_USA[covid_19_USA['State'] != 'Diamond Princess']
#covid_19_USA = covid_19_USA[covid_19_USA['State'] != 'Guam']

formatted_text('***USA Numbers -***')

# Data Glimpse
covid_19_USA.head()

***USA Numbers -***

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
98,Washington,US,47.4009,-121.4905,2020-01-22,0,0,0
99,New York,US,42.1657,-74.9481,2020-01-22,0,0,0
100,California,US,36.1162,-119.6816,2020-01-22,0,0,0
101,Massachusetts,US,42.2302,-71.5301,2020-01-22,0,0,0
104,Georgia,US,33.0406,-83.6431,2020-01-22,0,0,0


In [53]:
USA_statewise_data = covid_19_USA.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max()

USA_statewise_data['State'] = USA_statewise_data.index
USA_statewise_data.index = np.arange(1, len(USA_statewise_data.State.unique().tolist())+1)

USA_statewise_data = USA_statewise_data[['State','Confirmed', 'Deaths', 'Recovered']]

USA_locations = covid_19_USA[['State','Lat','Long']]
USA_locations.drop_duplicates(keep='first', inplace=True)

USA_statewise_data = pd.merge(USA_statewise_data, USA_locations, on='State')

formatted_text('***Country wise numbers of ''Confirmed'', ''Deaths'', ''Recovered'' Cases***')

USA_statewise_data.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



***Country wise numbers of Confirmed, Deaths, Recovered Cases***

,State,Confirmed,Deaths,Recovered,Lat,Long
0,Alabama,83,0,0,32.3182,-86.9023
1,Alaska,12,0,0,61.3707,-152.4044
2,Arizona,78,0,1,33.7298,-111.4312
3,Arkansas,96,0,0,34.9697,-92.3731
4,California,1177,23,6,36.1162,-119.6816


In [54]:
us_lat = 45
us_lon = -115

formatted_text('***Click on the pin to veiw details stats***')

USAMap = folium.Map(location=[us_lat, us_lon], zoom_start=4, tiles='cartodbpositron')

for lat, long, confirmed, deaths, recovered, state in zip(USA_statewise_data['Lat'],
                                                           USA_statewise_data['Long'],
                                                           USA_statewise_data['Confirmed'],
                                                           USA_statewise_data['Deaths'],
                                                           USA_statewise_data['Recovered'], 
                                                           USA_statewise_data['State']):
    
    if (deaths == 0):
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Total Confirmed:</strong> ' + str(confirmed) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='darkblue',icon='info-sign'), color='rgb(55, 83, 109)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(55, 83, 109)').add_to(USAMap)
    else:
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Total Confirmed:</strong> ' + str(confirmed) + '<br>'
                                 '<strong>Deaths:</strong> ' + str(int(deaths)) + '<br>'
                                 '<strong>Recovered:</strong> ' + str(int(recovered)) + '<br>')
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(26, 118, 255)').add_to(USAMap)
    
    
USAMap

***Click on the pin to veiw details stats***

In [55]:
temp_USA = USA_statewise_data.copy()
temp_USA['Country'] = 'United States of America' # to have a single root

fig = px.treemap(temp_USA, 
                 path=["Country", "State"], values="Confirmed", height=700,
                 title='Total Confirmed Cases In USA',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

fig = px.treemap(temp_USA, 
                 path=["Country", "State"], values="Deaths", height=700,
                 title='Total Reported Deaths In USA',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

**Washington, Newyork, California, Georgia, Louisinia are amongst the most affected states in the US**

## 3. Brazil

Data Credits - https://www.kaggle.com/unanimad/corona-virus-brazil <br></br>
Brazil Geo Data - https://www.kaggle.com/delimixx/geobrazil

In [56]:
# Brazil Data
covid_19_Brazil = covid_19[covid_19['Country'] == 'Brazil']
covid_19_Brazil.head()

,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered
35,Brazil,Brazil,-14.235,-51.9253,2020-01-22,0,0,0
321,Brazil,Brazil,-14.235,-51.9253,2020-01-23,0,0,0
607,Brazil,Brazil,-14.235,-51.9253,2020-01-24,0,0,0
893,Brazil,Brazil,-14.235,-51.9253,2020-01-25,0,0,0
1179,Brazil,Brazil,-14.235,-51.9253,2020-01-26,0,0,0


In [57]:
# Brazil Data
covid_Brazil = pd.read_csv("../input/corona-virus-brazil/brazil_covid19.csv")
#covid_Brazil
brazil_statewise_data = covid_Brazil.groupby(['state'])['suspects', 'cases','refuses', 'deaths'].max()
#brazil_statewise_data

brazil_statewise_data['state'] = brazil_statewise_data.index
brazil_statewise_data.index = np.arange(1, len(brazil_statewise_data.state.unique().tolist())+1)
brazil_statewise_data = brazil_statewise_data[['state', 'suspects', 'cases','refuses', 'deaths']]

geoBrazil = pd.read_csv("../input/brazilianstates/states.csv")
brazilian_statewise_data = pd.merge(brazil_statewise_data, geoBrazil, on='state')

brazilian_statewise_data.drop(['uf', 'population'], axis=1, inplace=True)
brazilian_statewise_data

,state,suspects,cases,refuses,deaths,latitude,longitude
0,Acre,8,4,0,0,-8.77,-70.55
1,Alagoas,7,4,29,0,-9.62,-36.82
2,Amapá,64,0,0,0,1.41,-51.77
3,Amazonas,7,3,42,0,-3.47,-65.10
4,Bahia,9,30,443,0,-13.29,-41.71
5,Ceará,6,24,118,0,-5.20,-39.53
6,Distrito Federal,82,84,134,0,-15.83,-47.86
7,Espírito Santo,71,13,148,0,-19.19,-40.34
8,Goiás,7,15,98,0,-15.98,-49.86
9,Maranhão,6,0,22,0,-5.42,-45.44


In [58]:
brazil_lat = -14.235
brazil_lon = -51.9253

formatted_text('***Click on the pin to veiw details stats***')

BrazilMap = folium.Map(location=[brazil_lat, brazil_lon], zoom_start=4, tiles='cartodbpositron')

for lat, long, suspects, confirmed, deaths, state in zip(brazilian_statewise_data['latitude'],
                                                           brazilian_statewise_data['longitude'],
                                                           brazilian_statewise_data['suspects'], 
                                                           brazilian_statewise_data['cases'],
                                                           brazilian_statewise_data['deaths'],
                                                           brazilian_statewise_data['state']):
    
    if (deaths == 0):
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Total Suspected:</strong> ' + str(suspects) + '<br>'
                                 '<strong>Confirmed Cases:</strong> ' + str(int(confirmed)) + '<br>'
                                 '<strong>Deaths Reported:</strong> ' + str(int(deaths)) + '<br>')
                    , icon=folium.Icon(color='darkblue',icon='info-sign'), color='rgb(55, 83, 109)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(55, 83, 109)').add_to(BrazilMap)
    else:
        folium.Marker(location=[lat, long]
                    , popup = ('<strong>nCov Numbers:</strong> ' + '<br>' + 
                               
                                 '<strong>State:</strong> ' + str(state).capitalize() + '<br>'
                                 '<strong>Total Suspected:</strong> ' + str(suspects) + '<br>'
                                 '<strong>Confirmed Cases:</strong> ' + str(int(confirmed)) + '<br>'
                                 '<strong>Deaths Reported:</strong> ' + str(int(deaths)) + '<br>')
                    , icon=folium.Icon(color='red', icon='info-sign'), color='rgb(26, 118, 255)'
                    , tooltip = str(state).capitalize(), fill_color='rgb(26, 118, 255)').add_to(BrazilMap)
    
    
BrazilMap

***Click on the pin to veiw details stats***

In [59]:
temp_Brazil = brazil_statewise_data.copy()
temp_Brazil['Country'] = 'Brazil' # to have a single root

fig = px.treemap(temp_Brazil, 
                 path=["Country", "state"], values="cases", height=700,
                 title='Total Confirmed Cases In Brazil',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

fig = px.treemap(temp_Brazil, 
                 path=["Country", "state"], values="deaths", height=700,
                 title='Total Reported Deaths In Brazil',
                 color_discrete_sequence = px.colors.qualitative.Prism)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

**Sao Polo has been the worst affected state in Brazil reporting a majority of 56% of the cases so far.**

### *More to be continued...*

# Data Insights:

`Covid-19: The new coronavirus disease now officially has a name.`
Read more: https://www.newscientist.com/article/2233218-covid-19-the-new-coronavirus-disease-now-officially-has-a-name/#ixzz6DpiKd2Ee

1. China is the worst affected where the numbers continue to rise.


2. Hubei is the worst affected State in China where the virus is believed to be originated.


3. The virus has spread to other countries as well. This could be due to people being present in China/Hubei as the time of the outbreak and in the event of their return to home 
   country unknowingly carried the virus along.


4. There have been many confirmed cases reported outside of Hubei. **Zhejiang, Guangdong, Henan, Hunan** beng the top four.


5. The recovery from the virus outside of Hubei has not been fast. The virus did not match any other known virus. This raised concern because when a virus is new, we do not know how it 
   affects people. There were no existing medications available. However, A team of doctors in Thailand have seen some apparent success treating Coronavirus with drug cocktail. The 
   doctors combined the anti-flu drug oseltamivir with lopinavir and ritonavir, anti-virals used to treat HIV, Kriengsak said, adding the ministry was awaiting research results to 
   prove the findings.

Read more at: https://economictimes.indiatimes.com/news/international/world-news/thailand-sees-apparent-success-treating-virus-with-drug-cocktail/articleshow/73879572.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst

6. Rest of World has reported cases from 27 different countries outsdie China. Most of the cases have been reported from Thailand, Singapore and Japan.


7. Honkong, Australia, South Korea have also reported more than 10 cases.


8. India has reported 3 cases so far. 

9. There have been countries where all the reported cases have been recovered. Such countries are **India, Belgium, Spain, Finland, Cambodia, Sri Lanka, Nepal, Russia**.



**This notebook is a continuation of https://www.kaggle.com/sauravmishra1710/covid-19-sars-cov-2-a-geo-statistical-analysis/**